### USE PYG ENVIRONMENT!!!!

In [1]:
# !pip install torch


In [2]:
# Install required packages
import os, torch

# os.environ['TORCH'] = torch.__version__
# !pip install pyg-lib -f https://data.pyg.org/whl/nightly/torch-${TORCH}.html
# !pip install git+https://github.com/pyg-team/pytorch_geometric.git

# !pip install sentence_transformers
# !pip3 install fuzzywuzzy[speedup]
# !pip install captum
# !pip install torch-sparse
# !pip install torch-scatter 


In [3]:
### load graph data 
import pickle

with open('../data/graphs/graph.pkl', 'rb') as f:
    data = pickle.load(f)


/Users/christianrasmussen/Documents/thesis/.conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
### create edge sampling loader
from torch_geometric.data import DataLoader
import torch_geometric.transforms as T
import numpy as np

train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1)(data)


In [ ]:
from torch_geometric.loader import LinkNeighborLoader

train_loader = LinkNeighborLoader(
    train_data,
    # Sample 30 neighbors for each node for 2 iterations
    num_neighbors=[10] * 2,
    # Use a batch size of 128 for sampling training nodes
    batch_size=128,
)

val_loader = LinkNeighborLoader(
    val_data,
    num_neighbors=[10] * 2,
    batch_size=128,
)

test_loader = LinkNeighborLoader(
    test_data,
    num_neighbors=[10] * 2,
    batch_size=128,
)

### create model
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv, to_hetero

class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = torch.nn.Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = torch.nn.Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = self.lin1(z).relu()
        z = self.lin2(z)
        return z.view(-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.decoder = EdgeDecoder(hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Model(hidden_channels=32)

print(model)


AttributeError: 'GlobalStorage' object has no attribute 'metadata'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### train model
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = F.mse_loss(out, data.y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_nodes

    return total_loss / len(train_loader)

### train model
for epoch in range(1, 101):
    loss = train()
    print('Epoch: {:03d}, Loss: {:.4f}'.format(epoch, loss))
